# Установка и настройка Pyspark

*Разбор данного раздела:* https://youtu.be/sOTh6klME_M?t=4631

In [ ]:
!wget http://mirror.klaus-uwe.me/apache/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz

--2021-02-01 18:43:01--  http://mirror.klaus-uwe.me/apache/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
Resolving mirror.klaus-uwe.me (mirror.klaus-uwe.me)... 94.130.201.139, 2a01:4f8:13b:228c::2
Connecting to mirror.klaus-uwe.me (mirror.klaus-uwe.me)|94.130.201.139|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233333392 (223M) [application/octet-stream]
Saving to: ‘spark-2.4.7-bin-hadoop2.7.tgz’

spark-2.4.7-bin-had 100%[===================>] 222.52M  30.0MB/s    in 8.0s    

2021-02-01 18:43:10 (27.8 MB/s) - ‘spark-2.4.7-bin-hadoop2.7.tgz’ saved [233333392/233333392]



In [ ]:
# !tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

openjdk version "1.8.0_282"
OpenJDK Runtime Environment (build 1.8.0_282-8u282-b08-0ubuntu1~20.04-b08)
OpenJDK 64-Bit Server VM (build 25.282-b08, mixed mode)


In [ ]:
!ls /usr/lib/jvm/java-8-openjdk-amd64

ASSEMBLY_EXCEPTION  docs     jre  man	   THIRD_PARTY_README
bin		    include  lib  src.zip


In [ ]:
import os
# Задаем окружение
# Указываем переменные окружения для findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-2.4.7-bin-hadoop2.7"

Инциализируем pyspark из директории с библиотекой

In [ ]:
import findspark
findspark.init()

Запускаем Spark-сессию

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

# PRO

In [ ]:
Взял примеры классификации из документации.
Первый классифкиатор разбил по ячейкам с выводом промежуточных значений - для собственного понимания того,
как обрабатываются данные

Оставшиеся модели аналогичны, поэтому выполнял код срзу в одной ячейке.

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier, RandomForestClassifier, MultilayerPerceptronClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, IndexToString
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


## GBTClassifier

In [ ]:
# Load and parse the data file, converting it to a DataFrame.
data = spark.read.format("libsvm").load("sample_libsvm_data.txt")


In [ ]:
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [ ]:

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)

In [ ]:

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

In [ ]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])
print(trainingData.show())

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[95,96,97,12...|
|  0.0|(692,[100,101,102...|
|  0.0|(692,[121,122,123...|
|  0.0|(692,[122,123,124...|
|  0.0|(692,[122,123,148...|
|  0.0|(692,[123,124,125...|
|  0.0|(692,[123,124,125...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[125,126,127...|
|  0.0|(692,[126,127,128...|
|  0.0|(692,[126,127,128...|
|  0.0|(692,[126,127,128...|
|  0.0|(692,[126,127,128...|
|  0.0|(692,[126,127,128...|
|  0.0|(692,[126,127,128...|
|  0.0|(692,[127,128,129...|
|  0.0|(692,[127,128,129...|
+-----+--------------------+
only showing top 20 rows

None


In [ ]:
# Train a GBT model.
gbt = GBTClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", maxIter=10)

In [ ]:
# Chain indexers and GBT in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, gbt])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

In [ ]:
predictions.show()

+-----+--------------------+------------+--------------------+--------------------+--------------------+----------+
|label|            features|indexedLabel|     indexedFeatures|       rawPrediction|         probability|prediction|
+-----+--------------------+------------+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[98,99,100,1...|         1.0|(692,[98,99,100,1...|[-1.2448727431634...|[0.07658019351661...|       1.0|
|  0.0|(692,[123,124,125...|         1.0|(692,[123,124,125...|[-1.3259026792203...|[0.06587782434721...|       1.0|
|  0.0|(692,[124,125,126...|         1.0|(692,[124,125,126...|[-1.3259026792203...|[0.06587782434721...|       1.0|
|  0.0|(692,[124,125,126...|         1.0|(692,[124,125,126...|[-1.3259026792203...|[0.06587782434721...|       1.0|
|  0.0|(692,[126,127,128...|         1.0|(692,[126,127,128...|[-1.2505858528532...|[0.07577607990715...|       1.0|
|  0.0|(692,[126,127,128...|         1.0|(692,[126,127,128...|[-1.250585

In [ ]:


# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(30)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

gbtModel = model.stages[2]
print(gbtModel)  # summary only

+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       1.0|         1.0|(692,[98,99,100,1...|
|       1.0|         1.0|(692,[123,124,125...|
|       1.0|         1.0|(692,[124,125,126...|
|       1.0|         1.0|(692,[124,125,126...|
|       1.0|         1.0|(692,[126,127,128...|
|       1.0|         1.0|(692,[126,127,128...|
|       1.0|         1.0|(692,[234,235,237...|
|       0.0|         0.0|(692,[97,98,99,12...|
|       0.0|         0.0|(692,[100,101,102...|
|       0.0|         0.0|(692,[123,124,125...|
|       0.0|         0.0|(692,[124,125,126...|
|       0.0|         0.0|(692,[124,125,126...|
|       0.0|         0.0|(692,[125,126,127...|
|       0.0|         0.0|(692,[127,128,129...|
|       0.0|         0.0|(692,[127,128,129...|
|       0.0|         0.0|(692,[127,128,129...|
|       0.0|         0.0|(692,[127,128,154...|
|       0.0|         0.0|(692,[128,129,130...|
|       0.0| 

## ПРО 2. Другие метрики оценки модели

### F-measure

In [ ]:
evaluator2 = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="f1")
accuracy2 = evaluator2.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy2))

Test Error = 0.0349593


### weightedPrecision

In [ ]:
evaluator3 = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="weightedPrecision")
accuracy3 = evaluator3.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy3))

Test Error = 0.03125


## RandomForestClassifier

In [ ]:
# Load and parse the data file, converting it to a DataFrame.
data = spark.read.format("libsvm").load("sample_libsvm_data.txt")

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("predictedLabel", "label", "features").show(30)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

rfModel = model.stages[2]
print(rfModel)  # summary only

+--------------+-----+--------------------+
|predictedLabel|label|            features|
+--------------+-----+--------------------+
|           0.0|  0.0|(692,[123,124,125...|
|           0.0|  0.0|(692,[124,125,126...|
|           0.0|  0.0|(692,[125,126,127...|
|           0.0|  0.0|(692,[126,127,128...|
|           0.0|  0.0|(692,[126,127,128...|
|           0.0|  0.0|(692,[126,127,128...|
|           0.0|  0.0|(692,[126,127,128...|
|           0.0|  0.0|(692,[152,153,154...|
|           0.0|  0.0|(692,[154,155,156...|
|           0.0|  0.0|(692,[154,155,156...|
|           1.0|  1.0|(692,[100,101,102...|
|           1.0|  1.0|(692,[119,120,121...|
|           1.0|  1.0|(692,[123,124,125...|
|           1.0|  1.0|(692,[123,124,125...|
|           1.0|  1.0|(692,[124,125,126...|
|           1.0|  1.0|(692,[124,125,126...|
|           1.0|  1.0|(692,[124,125,126...|
|           1.0|  1.0|(692,[125,126,153...|
|           1.0|  1.0|(692,[125,126,153...|
|           1.0|  1.0|(692,[126,

## MultilayerPerceptronClassifier

In [ ]:
# Load training data
data = spark.read.format("libsvm")\
    .load("sample_multiclass_classification_data.txt")

# Split the data into train and test
splits = data.randomSplit([0.6, 0.4], 1234)
train = splits[0]
test = splits[1]

# specify layers for the neural network:
# input layer of size 4 (features), two intermediate of size 5 and 4
# and output of size 3 (classes)
layers = [4, 5, 4, 3]

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)

# train the model
model = trainer.fit(train)

# compute accuracy on the test set
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

Test set accuracy = 0.9019607843137255


In [ ]:
result.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(4,[0,1,2,3],[-0....|[-29.588369001638...|[2.63020383878084...|       2.0|
|  0.0|(4,[0,1,2,3],[-0....|[125.657894478296...|[1.0,1.4484875476...|       0.0|
|  0.0|(4,[0,1,2,3],[-0....|[126.190155254739...|[1.0,5.1578089761...|       0.0|
|  0.0|(4,[0,1,2,3],[-0....|[-26.984478255346...|[4.23003198458660...|       2.0|
|  0.0|(4,[0,1,2,3],[-0....|[-29.588369001638...|[2.63020383878084...|       2.0|
|  0.0|(4,[0,1,2,3],[-1....|[-29.588368732563...|[2.63020459374897...|       2.0|
|  0.0|(4,[0,1,2,3],[0.1...|[126.190175711705...|[1.0,5.1572549882...|       0.0|
|  0.0|(4,[0,1,2,3],[0.2...|[126.190175994586...|[1.0,5.1572473280...|       0.0|
|  0.0|(4,[0,1,2,3],[0.3...|[126.190175994586...|[1.0,5.1572473280...|       0.0|
|  0.0|(4,[0,1,2